In [42]:
import openai

from fp_dataset_artifacts.utils import init_openai, get_qa_response
from fp_dataset_artifacts.anli import map_finetune_train
from datasets import list_datasets, load_dataset, list_metrics, load_metric, concatenate_datasets

init_openai()

data = load_dataset('anli')
data

Reusing dataset anli (/home/x/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)


  0%|          | 0/9 [00:00<?, ?it/s]

DatasetDict({
    train_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 16946
    })
    dev_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r1: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 45460
    })
    dev_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    test_r2: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 1000
    })
    train_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 100459
    })
    dev_r3: Dataset({
        features: ['uid', 'premise', 'hypothesis', 'label', 'reason'],
        num_rows: 12

In [4]:
train_r1 = data['train_r1'].map(map_finetune_train)
train_r1

  0%|          | 0/16946 [00:00<?, ?ex/s]

Dataset({
    features: ['uid', 'premise', 'hypothesis', 'label', 'reason', 'prompt', 'completion'],
    num_rows: 16946
})

In [8]:
def get_examples(exs):
    prompts = exs['prompt']
    completions = exs['completion']
    examples = [p + c for p, c in zip(prompts, completions)]
    return '\n\n'.join(examples)

examples = get_examples(train_r1[:5])
print(examples)

Premise: The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In operation since 1953, the system presently comprises four urban routes.

Hypothesis: The trolleybus system has over 2 urban routes

Label: Entailment


Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.

Hypothesis: Sharron Macready was a popular character through the 1980's.

Label: Neutral


Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series 

In [12]:
query = train_r1[6]['prompt']
print(query)
print('expected text:', train_r1[6]['completion'])

Premise: The Centralia Massacre was an incident during the American Civil War in which twenty-four unarmed Union soldiers were captured and executed at Centralia, Missouri on September 27, 1864 by the pro-Confederate guerrilla leader William T. Anderson. Future outlaw Jesse James was among the guerrillas.

Hypothesis: Jesse James was a guerrilla in the Union army during the American Civil War.

Label: 
expected text: Contradiction



In [15]:
INTRO = (
    'I am a highly intelligent natural language inference bot. '
    + 'If you give me a pair of premise and hypothesis, '
    + 'I will tell you whether the hypothesis is entailment, neutral, or contradiction.'
)
print(INTRO)

I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.


In [23]:
prompt = f'{INTRO}\n\n{examples}\n\n{query}'
print(prompt)

I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In operation since 1953, the system presently comprises four urban routes.

Hypothesis: The trolleybus system has over 2 urban routes

Label: Entailment


Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.

Hypothesis: Sharron Macready was a popular character through the 1980's.

Label: Neutral


Premise: Alexandra Le

In [24]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=['\n']
)
response

<OpenAIObject text_completion id=cmpl-4BwHSZuFiOrGG44uWSdYwWC9h16ZR at 0x7f112f5f64a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638750986,
  "id": "cmpl-4BwHSZuFiOrGG44uWSdYwWC9h16ZR",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [25]:
def get_prompt(x, exs):
    examples = get_examples(exs)
    query = x['prompt']
    prompt = f'{INTRO}\n\n{examples}\n\n{query}'
    return prompt

prompt = get_prompt(train_r1[7], train_r1[100:105])
print(prompt)

I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Porl King played guitar for a band named Rosetta Stone

Label: Entailment


Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is 

In [26]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=['\n']
)
response

<OpenAIObject text_completion id=cmpl-4BwHckHUEFc5P861b4cfvCxtWySTj at 0x7f112ce92400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638750996,
  "id": "cmpl-4BwHckHUEFc5P861b4cfvCxtWySTj",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [28]:
# Make the answer more natural, so that the model is more likely to generate the text.
prompt = """I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Porl King played guitar for a band named Rosetta Stone

The hypothesis entails the premise.


Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.

Hypothesis: Love in Nepal was released before July 4th, 2004.

The hypothesis entails the premise.


Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.

Hypothesis: Bastedo didn't keep any pets because of her views on animal rights.

The hypothesis and premise are neutral.


Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Rosetta Stone broke up.

The hypothesis entails the premise.


Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: King has a career in digital audio.

The hypothesis entails the premise.


Premise: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.

Hypothesis: The Other One is the third studio album made by Fleetwood Mac.

The hypothesis contradicts the premise.


Premise: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".

Hypothesis: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

"""
print(prompt)

I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Porl King played guitar for a band named Rosetta Stone

The hypothesis entails the premise.


Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the fem

In [29]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=['\n']
)
response

<OpenAIObject text_completion id=cmpl-4BwMPPnnGZdHmg7mBNt3rKz0nzULZ at 0x7f112c2eddb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638751293,
  "id": "cmpl-4BwMPPnnGZdHmg7mBNt3rKz0nzULZ",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [30]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwMonvsH8EBKbPN8vysUyQUfnCou at 0x7f112c2da270> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nPremise: Porl King, born 19 October 1967 on the Wirral Peninsula in England"
    }
  ],
  "created": 1638751318,
  "id": "cmpl-4BwMonvsH8EBKbPN8vysUyQUfnCou",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [31]:
# Rephrase the answer a bit to me more precise.
prompt = """I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Porl King played guitar for a band named Rosetta Stone

The above hypothesis and premise is an instance of entailment.


Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.

Hypothesis: Love in Nepal was released before July 4th, 2004.

The above hypothesis and premise is an instance of entailment.


Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.

Hypothesis: Bastedo didn't keep any pets because of her views on animal rights.

The above hypothesis and premise is an instance of neutral.


Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Rosetta Stone broke up.

The above hypothesis and premise is an instance of entailment.


Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: King has a career in digital audio.

The above hypothesis and premise is an instance of entailment.


Premise: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.

Hypothesis: The Other One is the third studio album made by Fleetwood Mac.

The above hypothesis and premise is an instance of contradiction.


Premise: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".

Hypothesis: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

"""
print(prompt)

I am a highly intelligent natural language inference bot. If you give me a pair of premise and hypothesis, I will tell you whether the hypothesis is entailment, neutral, or contradiction.

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.

Hypothesis: Porl King played guitar for a band named Rosetta Stone

The above hypothesis and premise is an instance of entailment.


Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal

In [32]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwOHU4XfaQ1NccEJAfSsfyUVJv85 at 0x7f112c2da0e0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nPremise: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch"
    }
  ],
  "created": 1638751409,
  "id": "cmpl-4BwOHU4XfaQ1NccEJAfSsfyUVJv85",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [33]:
# Simplify the language describing the task
prompt = """I am a highly intelligent natural language inference bot. If you give me a pair of sentences A and B, I will tell you whether it is an instance of entailment, neutral, or contradiction.

### 

A: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
B: Porl King played guitar for a band named Rosetta Stone

This is an instance of entailment.

### 

A: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.
B: Love in Nepal was released before July 4th, 2004.

This is an instance of entailment.

### 

A: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.
B: Bastedo didn't keep any pets because of her views on animal rights.

This is an instance of neutral.

### 

A: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
B: Rosetta Stone broke up.

This is an instance of entailment.

### 

A: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
B: King has a career in digital audio.

This is an instance of entailment.

### 

A: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.
B: The Other One is the third studio album made by Fleetwood Mac.

This is an instance of contradiction.

### 

A: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".
B: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

This is an instance of """
print(prompt)

I am a highly intelligent natural language inference bot. If you give me a pair of sentences A and B, I will tell you whether it is an instance of entailment, neutral, or contradiction.

### 

A: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
B: Porl King played guitar for a band named Rosetta Stone

This is an instance of entailment.

### 

A: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This

In [34]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwQujKhSwwAEtY6ROaNXXEfrC8Eo at 0x7f112c2daa40> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n### \n\nA: The Other One is the third solo album by former Fleetwood"
    }
  ],
  "created": 1638751572,
  "id": "cmpl-4BwQujKhSwwAEtY6ROaNXXEfrC8Eo",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [36]:
# Rephrase it as question answering task
prompt = """I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Hypothesis: Porl King played guitar for a band named Rosetta Stone

Q: What is the instance of the premise and hypothesis?
A: It is an instance of entailment.

### 

Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.
Hypothesis: Love in Nepal was released before July 4th, 2004.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of entailment.

### 

Premise: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.
Hypothesis: Bastedo didn't keep any pets because of her views on animal rights.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of neutral.

### 

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Hypothesis: Rosetta Stone broke up.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of entailment.

### 

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Hypothesis: King has a career in digital audio.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of entailment.

### 

Premise: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.
Hypothesis: The Other One is the third studio album made by Fleetwood Mac.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of contradiction.

### 

Premise: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".
Hypothesis: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

Q: What is the instance of the premise and hypothesis?
A: It is an instance of """
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Premise: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Hypothesis: Porl King played guitar for a band named Rosetta Stone

Q: What is the instance of the premise and hypothesis?
A: It is an instance of entailment.

### 

Premise: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback sin

In [37]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwZ3jRsJqU5Ny2HMcio28oj7OzOL at 0x7f112c2e8ef0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n### \n\nPremise: Porl King, born 19 October 1967 on the W"
    }
  ],
  "created": 1638752077,
  "id": "cmpl-4BwZ3jRsJqU5Ny2HMcio28oj7OzOL",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [38]:
# Don't use NLI language
prompt = """I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Porl King played guitar for a band named Rosetta Stone

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.
Sentence 2: Love in Nepal was released before July 4th, 2004.

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.
Sentence 2: Bastedo didn't keep any pets because of her views on animal rights.

Q: What is the relationship between the sentence 1 and 2?
A: They are not related at all.

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Rosetta Stone broke up.

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: King has a career in digital audio.

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.
Sentence 2: The Other One is the third studio album made by Fleetwood Mac.

Q: What is the relationship between the sentence 1 and 2?
A: They are in a disagreement.

### 

Sentence 1: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".
Sentence 2: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

Q: What is the relationship between the sentence 1 and 2?
A: """
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Porl King played guitar for a band named Rosetta Stone

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback s

In [39]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwczoKa0mGaQo2OfsERMAgh2OOV8 at 0x7f118c6d5c20> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n### \n\nSentence 1: Porl King, born 19 October 1967 on the"
    }
  ],
  "created": 1638752321,
  "id": "cmpl-4BwczoKa0mGaQo2OfsERMAgh2OOV8",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [40]:
# Add some variations
prompt = """I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Porl King played guitar for a band named Rosetta Stone

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback singer, the film has Nepali film actress Jharana Bajracharya and Richa Ghimire from Nepal and Flora Saini in the female lead. This is Sonu Nigam's second film as the male lead.
Sentence 2: Love in Nepal was released before July 4th, 2004.

Q: What is the relationship between the sentence 1 and 2?
A: These sentences agrees on the release date of Love in Nepal.

### 

Sentence 1: Alexandra Lendon Bastedo (9 March 1946 – 12 January 2014) was a British actress, best known for her role as secret agent Sharron Macready in the 1968 British espionage/science fiction adventure series "The Champions". She has been cited as a sex symbol of the 1960s and 1970s. Bastedo was a vegetarian and animal welfare advocate.
Sentence 2: Bastedo didn't keep any pets because of her views on animal rights.

Q: What is the relationship between the sentence 1 and 2?
A: It is hard to tell whether these sentences tell me anything notable.

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Rosetta Stone broke up.

Q: What is the relationship between the sentence 1 and 2?
A: These sentences are related.

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: King has a career in digital audio.

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: The Other One is the third solo album by former Fleetwood Mac guitarist Bob Welch. The track "Future Games" was first released on the Fleetwood Mac album of the same name in 1971. Members of Welch's backing band also make songwriting contributions here though the majority of tracks are Welch's own.
Sentence 2: The Other One is the third studio album made by Fleetwood Mac.

Q: What is the relationship between the sentence 1 and 2?
A: They are in contradiction.

### 

Sentence 1: Rhodochiton is a genus of flowering plants within the family Plantaginaceae, native to southern Mexico and neighbouring Guatemala. They climb by means of twining leaf stalks. One of the three species, "Rhodochiton atrosanguineus", the purple bell vine, is grown as an ornamental plant. All three species are sometimes included in "Lophospermum".
Sentence 2: Rhodochiton atrosanguineus is a species of the family Plantaginaceae.

Q: What is the relationship between the sentence 1 and 2?
A: """
print(prompt)

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will respond with "Unknown".

### 

Sentence 1: Porl King, born 19 October 1967 on the Wirral Peninsula in England, first achieved public acclaim and success as the singer, songwriter and guitarist for the goth band Rosetta Stone from their inception in the late 1980s through to their demise in 1998. Since disbanding Rosetta Stone in 1998, he has pursued a career in digital audio and Pro Tools production.
Sentence 2: Porl King played guitar for a band named Rosetta Stone

Q: What is the relationship between the sentence 1 and 2?
A: They are in an agreement.

### 

Sentence 1: Love in Nepal is a 2004 Indian Hindi romantic thriller film directed by Rajat Mukherjee which was released on 6 March 2004. Unlike Sonu Nigam in the male lead, who is otherwise a well known Bollywood playback s

In [41]:
response = openai.Completion.create(
  engine='davinci',
  prompt=prompt,
  temperature=0.3,
  max_tokens=20,
  top_p=1,
  frequency_penalty=0.5,
  presence_penalty=0.0,
)
response

<OpenAIObject text_completion id=cmpl-4BwfuwKZPFCI84ZF7NzK24tWyC5eg at 0x7f112c1d4bd0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n### \n\nSentence 1: Rhodochiton is a genus of flowering plants within"
    }
  ],
  "created": 1638752502,
  "id": "cmpl-4BwfuwKZPFCI84ZF7NzK24tWyC5eg",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [43]:
get_qa_response(prompt)

<OpenAIObject text_completion id=cmpl-4BwjTuZDe4Y9JZg73OMIyJQYjIQjr at 0x7f112c2f1270> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638752723,
  "id": "cmpl-4BwjTuZDe4Y9JZg73OMIyJQYjIQjr",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}